### (1) Load the pretrained model

In [1]:
import sys
sys.path.insert(1, './dlite-v2-1_5b/')
from instruct_pipeline import InstructionTextGenerationPipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("dlite-v2-1_5b", padding_side="left")
model = AutoModelForCausalLM.from_pretrained("dlite-v2-1_5b", device_map="cuda", torch_dtype=torch.bfloat16)

generate_text = InstructionTextGenerationPipeline(model=model, tokenizer=tokenizer)

### (2) The REST server

In [ ]:
from flask import Flask, request, jsonify
import subprocess

app = Flask(__name__)

def req_length(text):
    tokens = tokenizer(text)
    tokens_length = len(tokens.attention_mask)
    print('LOG: Input length: ' + str(tokens_length))
    return tokens_length

def req_to_json(title, text):
    return jsonify({title : text})

def req_process(command, text):
    req = command + ' ' + text
    print('LOG: ' + req)
    return generate_text(req)

@app.route('/sum', methods=['POST'])
def sum_req():
    data = request.get_json()
    text = data.get('text', '')
    if(req_length(text) <= 1024):
        return req_to_json('response', req_process('Summarize in one sentence:', text))
    else:
        return req_to_json('error', 'Too long text. Limit is 1024 token.')

@app.route('/key', methods=['POST'])
def key_req():
    data = request.get_json()
    text = data.get('text', '')
    if(req_length(text) <= 1024):
        return req_to_json('response', req_process('Give keyword list:', text))
    else:
        return req_to_json('error', 'Too long text. Limit is 1024 token.')

@app.route('/title', methods=['POST'])
def title_req():
    data = request.get_json()
    text = data.get('text', '')
    if(req_length(text) <= 1024):
        return req_to_json('response', req_process('Give the most important word in the text:', text))
    else:
        return req_to_json('error', 'Too long text. Limit is 1024 token.')

@app.route('/length', methods=['POST'])
def length_req():
    data = request.get_json()
    text = data.get('text', '')
    ret = {'length': req_length(text)}
    return jsonify(ret)

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [09/May/2024 18:19:03] "POST /length HTTP/1.1" 200 -


LOG: Input length: 243
LOG: Input length: 243
LOG: Summarize in one sentence: Jupiter, the largest planet in our solar system, commands attention with its immense size and swirling clouds of gas. Named after the king of the Roman gods, Jupiter's colossal presence dominates the space around it. Its massive magnetic field creates a protective shield, deflecting harmful solar winds and radiation, making it a formidable guardian of the inner solar system. One of Jupiter's most iconic features is its Great Red Spot, a massive storm that has been raging for centuries. This colossal tempest is larger than Earth itself, a testament to the planet's tumultuous atmosphere. Beneath its thick layer of clouds lies a world of mystery, with a core composed of metallic hydrogen and rocky material. Jupiter boasts a diverse family of moons, each with its own unique characteristics. Europa, with its icy surface and subsurface ocean, tantalizes scientists with the possibility of extraterrestrial life. Gany

127.0.0.1 - - [09/May/2024 18:19:47] "POST /sum HTTP/1.1" 200 -


LOG: Input length: 243
LOG: Give keyword list: Jupiter, the largest planet in our solar system, commands attention with its immense size and swirling clouds of gas. Named after the king of the Roman gods, Jupiter's colossal presence dominates the space around it. Its massive magnetic field creates a protective shield, deflecting harmful solar winds and radiation, making it a formidable guardian of the inner solar system. One of Jupiter's most iconic features is its Great Red Spot, a massive storm that has been raging for centuries. This colossal tempest is larger than Earth itself, a testament to the planet's tumultuous atmosphere. Beneath its thick layer of clouds lies a world of mystery, with a core composed of metallic hydrogen and rocky material. Jupiter boasts a diverse family of moons, each with its own unique characteristics. Europa, with its icy surface and subsurface ocean, tantalizes scientists with the possibility of extraterrestrial life. Ganymede, the largest moon in the s